In [ ]:
!7z x Dataset.7z

In [ ]:
!pip install miditoolkit mido pandas seaborn matplotlib

In [4]:
import miditoolkit
from mido import MidiFile, MidiTrack, MetaMessage, Message
import os
import pandas as pd
import json
import numpy as np
from pathlib import Path
import torch

In [3]:
def extract_note_attributes(note):
    duration = max(60, round((note.end - note.start) / 60) * 60)
    return (note.pitch, note.start, duration)


def insert_silence_tokens(note_events):
    result = []
    prev_end = None

    for pitch, start, duration in note_events:
        if prev_end is not None and start > prev_end:
            silence_duration = max(60, round((start - prev_end) / 60) * 60)
            result.append((0, silence_duration))

        result.append((pitch, duration))
        prev_end = start + duration

    return result


def extract_metadata(filename):
    midi = MidiFile(filename)
    metadata = {
        'ticks_per_beat': midi.ticks_per_beat,
        'time_signature': MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8),
        'key_signature': MetaMessage('key_signature', key='C'),
        'set_tempo': MetaMessage('set_tempo', tempo=500000)  # Default 120 BPM
    }

    for track in midi.tracks:
        for msg in track:
            if msg.type in metadata:
                metadata[msg.type] = msg

    return metadata


def encode_midi(filename):
    midi = miditoolkit.MidiFile(filename)
    note_events = [
        extract_note_attributes(note)
        for instrument in midi.instruments
        for note in instrument.notes
    ]

    tokens = insert_silence_tokens(note_events)
    return tokens


def tokens_to_ids(tokens, token_dict):
    return [token_dict[token] for token in tokens]


def ids_to_tokens(ids, id_to_token):
    return [id_to_token[_id] for _id in ids]


def decode_midi(notes, output_filename, metadata):
    mid = MidiFile(ticks_per_beat=metadata['ticks_per_beat'])
    track = MidiTrack()
    mid.tracks.append(track)

    # Add metadata and instrument info
    track.extend([
        metadata['time_signature'],
        metadata['key_signature'],
        metadata['set_tempo'],
        Message('program_change', channel=0, program=1, time=0),  # Instrument
        Message('control_change', channel=0, control=7, value=80, time=0)  # Volume
    ])

    current_time = 0
    for pitch, duration in notes:

        if pitch == 0:
            current_time += duration
        else:
            track.append(Message('note_on', note=pitch, velocity=100, time=current_time))
            track.append(Message('note_on', note=pitch, velocity=0, time=duration))
            current_time = 0

    mid.save(output_filename)


def decode_ids_to_midi(id_sequence, output_filename, metadata, id_to_token):
    tokens = ids_to_tokens(id_sequence, id_to_token)
    decode_midi(tokens, output_filename, metadata)



In [ ]:
DATASET_DIR = "/content/Dataset"
all_tokens = []
token_set = set()

for filename in os.listdir(DATASET_DIR):
    if filename.lower().endswith('.mid'):
        filepath = os.path.join(DATASET_DIR, filename)
        tokens = encode_midi(filepath)

        all_tokens.extend(tokens)
        token_set.update(tokens)

        print(f"Encoded {filename}")


In [7]:
print(len(all_tokens)) # 1641743 tokens in vocab

1641743


In [9]:
print(list(token_set)[:50]) # vocab in tuples
print(len(token_set)) # vocab size = 2788

[(36, 480), (69, 1560), (60, 1740), (51, 1920), (0, 2220), (44, 420), (0, 13740), (81, 6000), (71, 2280), (0, 59640), (79, 8520), (0, 36600), (86, 3720), (67, 180), (58, 360), (70, 2400), (75, 300), (83, 6540), (66, 480), (52, 2760), (57, 660), (0, 9660), (0, 21180), (0, 32700), (48, 840), (81, 1920), (39, 1020), (72, 2100), (84, 2340), (59, 1380), (56, 780), (45, 3660), (67, 7620), (47, 960), (50, 1560), (83, 2640), (36, 3840), (74, 2820), (0, 5760), (76, 120), (88, 360), (79, 540), (82, 2760), (40, 1860), (78, 840), (67, 3720), (69, 1020), (60, 1200), (51, 1380), (0, 1680)]
2788


In [11]:
token_dict = {token: idx for idx, token in enumerate(sorted(token_set))}
print(list(token_dict.items())[:50])
print(len(token_dict))

[((0, 60), 0), ((0, 120), 1), ((0, 180), 2), ((0, 240), 3), ((0, 300), 4), ((0, 360), 5), ((0, 420), 6), ((0, 480), 7), ((0, 540), 8), ((0, 600), 9), ((0, 660), 10), ((0, 720), 11), ((0, 780), 12), ((0, 840), 13), ((0, 900), 14), ((0, 960), 15), ((0, 1020), 16), ((0, 1080), 17), ((0, 1140), 18), ((0, 1200), 19), ((0, 1260), 20), ((0, 1320), 21), ((0, 1380), 22), ((0, 1440), 23), ((0, 1500), 24), ((0, 1560), 25), ((0, 1620), 26), ((0, 1680), 27), ((0, 1740), 28), ((0, 1800), 29), ((0, 1860), 30), ((0, 1920), 31), ((0, 1980), 32), ((0, 2040), 33), ((0, 2100), 34), ((0, 2160), 35), ((0, 2220), 36), ((0, 2280), 37), ((0, 2340), 38), ((0, 2400), 39), ((0, 2460), 40), ((0, 2520), 41), ((0, 2580), 42), ((0, 2640), 43), ((0, 2700), 44), ((0, 2760), 45), ((0, 2820), 46), ((0, 2880), 47), ((0, 2940), 48), ((0, 3000), 49)]
2788


In [12]:
# Create token-to-ID mapping
token2id = {token: idx for idx, token in enumerate(sorted(token_set))}
id2token = {idx: token for token, idx in token2id.items()}  # Optional: for decoding

In [13]:
all_token_ids = tokens_to_ids(all_tokens, token2id)

In [15]:
print(len(all_token_ids)) # All tokens from their IDs
print(all_token_ids[:50])

1641743
[2372, 0, 2464, 0, 2370, 0, 2462, 0, 2276, 0, 2462, 0, 2372, 0, 2011, 0, 2222, 0, 2278, 0, 2220, 0, 2276, 0, 2107, 0, 2276, 0, 2227, 2372, 0, 2464, 0, 2370, 0, 2462, 0, 2276, 0, 2462, 0, 2372, 0, 2614, 0, 2517, 0, 2464, 0, 2553]


In [16]:
import torch

data = torch.tensor(all_token_ids, dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1641743]) torch.int64
tensor([2372,    0, 2464,    0, 2370,    0, 2462,    0, 2276,    0, 2462,    0,
        2372,    0, 2011,    0, 2222,    0, 2278,    0, 2220,    0, 2276,    0,
        2107,    0, 2276,    0, 2227, 2372,    0, 2464,    0, 2370,    0, 2462,
           0, 2276,    0, 2462,    0, 2372,    0, 2614,    0, 2517,    0, 2464,
           0, 2553,    0, 2614,    0, 2558, 2674,    0, 2674,    0, 2464,    0,
        2614,    0, 2551,    0, 2612,    0, 2551,    0, 2462,    0, 2370,    0,
        2329,    0, 2222,    0, 2464,    0, 2222,    0, 2331,    0, 2220,    0,
        2107,    0, 2372,    0, 2011,    0, 2011,    0, 2011,    0, 2011,    0,
        1958,    0, 1846,    0])


Splitting MIDI files into training and validation sets with a 0.1:0.9 ratio

In [17]:
# split the data into training and testing data with a ration of 0.9:0.1
n = int(0.9*len(data))
train_data = data[:n]
test_data = data[n:]

print(len(train_data)) # 0.9 * 1641743 = 1477568
print(len(test_data)) # 0.1 * 1641743 = 164175

1477568
164175


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else test_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [ ]:
from torch.utils.data import Dataset

class MIDITokenDataset(Dataset):
    def __init__(self, data, block_size):
        self.data = data
        self.block_size = block_size

    def __len__(self):
        return len(self.data) - self.block_size

    def __getitem__(self, idx):
        x = self.data[idx : idx + self.block_size]
        y = self.data[idx + 1 : idx + 1 + self.block_size]
        return {"input_ids": x, "labels": y}


In [ ]:
from torch.utils.data import DataLoader

block_size = 128
batch_size = 32
n_embd = 128
n_head = 4
n_layer = 4
dropout = 0.1

eval_iters = 200

train_dataset = MIDITokenDataset(train_data, block_size=block_size)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Head(nn.Module):
    def __init__(self, head_size, n_embd, block_size, dropout=0.1):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        v = self.value(x)
        wei = q @ k.transpose(-2, -1) / (k.size(-1) ** 0.5)

        wei = wei.masked_fill(self.tril[:T, :T] == 0, float("-inf"))
        wei = F.softmax(wei, dim=-1)
        return self.dropout(wei @ v)

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, n_embd, block_size, dropout=0.1):
        super().__init__()
        head_size = n_embd // num_heads
        self.heads = nn.ModuleList([Head(head_size, n_embd, block_size, dropout) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.dropout(self.proj(torch.cat([h(x) for h in self.heads], dim=-1)))

class FeedForward(nn.Module):
    def __init__(self, n_embd, dropout=0.1):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.GELU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self, n_embd, n_head, block_size, dropout=0.1):
        super().__init__()
        self.sa = MultiHeadAttention(n_head, n_embd, block_size, dropout)
        self.ffwd = FeedForward(n_embd, dropout)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class TransformerMIDILanguageModel(nn.Module):
    def __init__(self, vocab_size, n_embd=128, n_head=4, n_layer=6, block_size=128, dropout=0.1):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head, block_size, dropout) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
        self.block_size = block_size
        self.vocab_size = vocab_size

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=idx.device))

        x = self.blocks(tok_emb + pos_emb)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, self.vocab_size), targets.view(-1), ignore_index=-100)
        return logits, loss

    def generate(self, idx, max_new_tokens, temperature=1.0, top_k=None, eos_token_id=None):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -self.block_size:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :] / temperature

            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)

            if eos_token_id is not None and idx_next.item() == eos_token_id:
                break

        return idx

In [ ]:
vocab_size = len(token2id)

model = TransformerMIDILanguageModel(
    vocab_size=vocab_size,
    n_embd=n_embd,
    n_head=n_head,
    n_layer=n_layer,
    block_size=block_size,
    dropout=dropout
).to(device)

torch.manual_seed(1337)

print(sum(p.numel() for p in model.parameters()) / 1e6, "M parameters")


1.524708 M parameters


In [ ]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            _, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [ ]:
learning_rate = 1e-4
max_iters = 20000
eval_interval = 100

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch('train')
    _, loss = model(xb, yb)

    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


step 0: train loss 3.3903, val loss 3.4575
step 100: train loss 3.3839, val loss 3.4364
step 200: train loss 3.3597, val loss 3.4281
step 300: train loss 3.3590, val loss 3.4236
step 400: train loss 3.3357, val loss 3.4017
step 500: train loss 3.3273, val loss 3.3985
step 600: train loss 3.3070, val loss 3.3865
step 700: train loss 3.2937, val loss 3.3872
step 800: train loss 3.2933, val loss 3.3660
step 900: train loss 3.2923, val loss 3.3597
step 1000: train loss 3.2810, val loss 3.3353
step 1100: train loss 3.2685, val loss 3.3419
step 1200: train loss 3.2644, val loss 3.3359
step 1300: train loss 3.2531, val loss 3.3153
step 1400: train loss 3.2480, val loss 3.3177
step 1500: train loss 3.2180, val loss 3.3188
step 1600: train loss 3.2186, val loss 3.3151
step 1700: train loss 3.2235, val loss 3.2924
step 1800: train loss 3.2229, val loss 3.2807
step 1900: train loss 3.1973, val loss 3.2842
step 2000: train loss 3.1988, val loss 3.2885
step 2100: train loss 3.2044, val loss 3.2668


In [ ]:
start_token = token2id[(0, 480)]  # or any common starting token
context = torch.tensor([[start_token]], dtype=torch.long, device=device)

generated_ids = model.generate(context, max_new_tokens=500)[0].tolist()

In [ ]:
print(generated_ids)

[7, 2226, 2070, 2121, 2070, 2113, 2070, 2121, 2113, 2226, 2384, 2376, 2226, 2113, 2234, 2121, 2023, 1876, 2113, 2226, 2282, 2226, 2113, 2250, 23, 2230, 2278, 2222, 2109, 1964, 1852, 11, 2476, 2376, 2282, 1964, 11, 2464, 2372, 2276, 2226, 2113, 7, 2468, 2370, 2220, 2464, 2066, 2109, 2070, 2224, 2107, 1848, 1960, 2222, 2109, 2222, 2226, 2109, 1733, 2278, 2222, 2113, 2019, 2011, 2222, 2109, 1733, 1848, 2011, 1733, 1848, 1848, 2011, 2011, 1962, 1845, 1733, 7, 2222, 2276, 2370, 2220, 2109, 1852, 1964, 1733, 1848, 1960, 1735, 1731, 1850, 1846, 1960, 2011, 1958, 2064, 2107, 2220, 2109, 2109, 2226, 2113, 2226, 2431, 2278, 2226, 11, 1850, 1958, 2064, 2109, 2220, 2329, 2222, 1846, 2107, 2220, 2278, 2222, 2111, 2219, 2328, 2372, 2335, 3, 2113, 2066, 1846, 1958, 2066, 2109, 2374, 2220, 2278, 2224, 2107, 2015, 7, 1850, 1958, 2011, 1848, 2464, 2372, 2462, 2370, 1958, 2276, 2372, 2464, 2464, 2468, 3, 2372, 2224, 2276, 2109, 1850, 2009, 3, 2466, 2370, 2329, 2374, 2220, 2107, 2226, 1960, 1585, 2370, 24

In [ ]:
metadata={
    "ticks_per_beat": 480,
    "time_signature": MetaMessage("time_signature", numerator=4, denominator=4),
    "key_signature": MetaMessage("key_signature", key="C"),
    "set_tempo": MetaMessage("set_tempo", tempo=500000)
}

decode_ids_to_midi(generated_ids, "generated_output.mid", metadata, id2token)